In [1]:
%autoawait

IPython autoawait is `on`, and set to use `asyncio`


In [2]:
import time
import asyncio
import flask
from flask import request

In [3]:
from aries_basic_controller.aries_controller import AriesAgentController
    
WEBHOOK_HOST = "0.0.0.0"
WEBHOOK_PORT = 8022
WEBHOOK_BASE = ""
ADMIN_URL = "http://data-agent:8021"

In [4]:
    # Based on the aca-py agent you wish to control
agent_controller = AriesAgentController(webhook_host=WEBHOOK_HOST, webhook_port=WEBHOOK_PORT,
                                       webhook_base=WEBHOOK_BASE, admin_url=ADMIN_URL, connections=True)

In [19]:
def data_messages_hook(payload):
    connection_id = payload["connection_id"]
    asyncio.get_event_loop().create_task(agent_controller.connections.send_message(connection_id, "THIS IS MY RESPONSE - DATA AGENT"))
    print("Handle data messages ", payload, connection_id)


data_message_listener = {
    "handler": data_messages_hook,
    "topic": "basicmessages"
}

In [6]:
loop = asyncio.get_event_loop()
loop.create_task(agent_controller.listen_webhooks())

<Task pending coro=<AriesAgentController.listen_webhooks() running at /opt/conda/lib/python3.7/site-packages/aries_basic_controller/aries_controller.py:42>>

In [20]:
agent_controller.register_listeners([data_message_listener], defaults=True)

In [8]:
# Create Invitation
invite = await agent_controller.connections.create_invitation(alias="Will")
print(invite)

{'connection_id': '1ef4764f-b40c-498f-a8ae-ead9637fbdd9', 'invitation': {'@type': 'did:sov:BzCbsNYhMrjHiqZDTUASHg;spec/connections/1.0/invitation', '@id': '177c7dfb-2adc-4f06-a306-dfae75c6574e', 'label': 'Bob', 'serviceEndpoint': 'http://172.17.0.1:8020', 'recipientKeys': ['4f8A5rUKLXFMHPtxXCMZN6MCWDsSAB4d3TC7UMcf4Ree']}, 'invitation_url': 'http://172.17.0.1:8020?c_i=eyJAdHlwZSI6ICJkaWQ6c292OkJ6Q2JzTlloTXJqSGlxWkRUVUFTSGc7c3BlYy9jb25uZWN0aW9ucy8xLjAvaW52aXRhdGlvbiIsICJAaWQiOiAiMTc3YzdkZmItMmFkYy00ZjA2LWEzMDYtZGZhZTc1YzY1NzRlIiwgImxhYmVsIjogIkJvYiIsICJzZXJ2aWNlRW5kcG9pbnQiOiAiaHR0cDovLzE3Mi4xNy4wLjE6ODAyMCIsICJyZWNpcGllbnRLZXlzIjogWyI0ZjhBNXJVS0xYRk1IUHR4WENNWk42TUNXRHNTQUI0ZDNUQzdVTWNmNFJlZSJdfQ=='}


In [9]:
# Print out accepted Invite and Researcher ID
print("Data Owner ID", invite["connection_id"])
data_connection_id = invite["connection_id"]

Data Owner ID 1ef4764f-b40c-498f-a8ae-ead9637fbdd9


In [10]:
# Accept Request for Invite created
connection = await agent_controller.connections.accept_request(data_connection_id)
print("ACCEPT REQUEST")
print(connection)

True
ACCEPT REQUEST
{'invitation_mode': 'once', 'routing_state': 'none', 'updated_at': '2020-06-25 15:10:30.486591Z', 'their_label': 'Alice', 'their_did': '2Qbiw2yuNDpxtX8CHwKCze', 'initiator': 'self', 'accept': 'manual', 'invitation_key': '4f8A5rUKLXFMHPtxXCMZN6MCWDsSAB4d3TC7UMcf4Ree', 'created_at': '2020-06-25 15:09:40.096305Z', 'state': 'response', 'connection_id': '1ef4764f-b40c-498f-a8ae-ead9637fbdd9', 'my_did': '48RxqgaB7ywbPqMpTqxg2Z'}


In [11]:
trust_ping = await agent_controller.connections.trust_ping(data_connection_id, "hello")
print(trust_ping)

{'thread_id': 'ecef5225-d1f0-4d88-9290-2754c65d98fb'}


In [12]:
connection = await agent_controller.connections.get_connection(data_connection_id)
print(connection)

{'invitation_mode': 'once', 'routing_state': 'none', 'updated_at': '2020-06-25 15:10:42.606939Z', 'their_label': 'Alice', 'their_did': '2Qbiw2yuNDpxtX8CHwKCze', 'initiator': 'self', 'accept': 'manual', 'invitation_key': '4f8A5rUKLXFMHPtxXCMZN6MCWDsSAB4d3TC7UMcf4Ree', 'created_at': '2020-06-25 15:09:40.096305Z', 'state': 'active', 'connection_id': '1ef4764f-b40c-498f-a8ae-ead9637fbdd9', 'my_did': '48RxqgaB7ywbPqMpTqxg2Z'}


In [18]:
message = await agent_controller.connections.send_message(data_connection_id,"hello from data owner world! WOOPWOOP")
print("BASIC MESSAGE - DATA -> RESEARCH")
print(message)

Error handling request
Traceback (most recent call last):
  File "/opt/conda/lib/python3.7/site-packages/aiohttp/web_protocol.py", line 418, in start
    resp = await task
  File "/opt/conda/lib/python3.7/site-packages/aiohttp/web_app.py", line 458, in _handle
    resp = await handler(request)
  File "/opt/conda/lib/python3.7/site-packages/aries_basic_controller/aries_controller.py", line 53, in _receive_webhook
    await self.handle_webhook(topic, payload)
  File "/opt/conda/lib/python3.7/site-packages/aries_basic_controller/aries_controller.py", line 59, in handle_webhook
    pub.sendMessage(topic, payload=payload)
  File "/opt/conda/lib/python3.7/site-packages/pubsub/core/publisher.py", line 216, in sendMessage
    topicObj.publish(**msgData)
  File "/opt/conda/lib/python3.7/site-packages/pubsub/core/topicobj.py", line 452, in publish
    self.__sendMessage(msgData, topicObj, msgDataSubset)
  File "/opt/conda/lib/python3.7/site-packages/pubsub/core/topicobj.py", line 482, in __sendM

BASIC MESSAGE - DATA -> RESEARCH
{}


In [15]:
app = Flask(__name__)

@app.route('/gridhook', methods=['POST'])
def demo():
    if request.method == 'POST':
        content = request.json
        message = await agent_controller.connections.send_message(data_connection_id, content)

SyntaxError: 'await' outside async function (<ipython-input-15-1e06e3d35104>, line 10)